## Pre procesamiento de los datos

In [1]:
# importando librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
# cargar el dataset
datos = pd.read_csv('us_perm_visas.csv',low_memory=False)

In [3]:
# mostrar las primeras 3 filas
datos.head(3)

,add_these_pw_job_title_9089,agent_city,agent_firm_name,agent_state,application_type,case_no,case_number,case_received_date,case_status,class_of_admission,...,ri_pvt_employment_firm_to,ri_us_workers_considered,schd_a_sheepherder,us_economic_sector,wage_offer_from_9089,wage_offer_to_9089,wage_offer_unit_of_pay_9089,wage_offered_from_9089,wage_offered_to_9089,wage_offered_unit_of_pay_9089
0,NaN,NaN,NaN,NaN,PERM,A-07323-97014,NaN,NaN,Certified,J-1,...,NaN,NaN,NaN,IT,75629.0,NaN,yr,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,PERM,A-07332-99439,NaN,NaN,Denied,B-2,...,NaN,NaN,NaN,Other Economic Sector,37024.0,NaN,yr,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,PERM,A-07333-99643,NaN,NaN,Certified,H-1B,...,NaN,NaN,NaN,Aerospace,47923.0,NaN,yr,NaN,NaN,NaN


### Pre-procesamiento de datos

In [4]:
def getInfodata(data): 
    print('Numero de postulaciones para obtener visa: ', len(data) )
    print('Numero de características: ', len(data.columns) )

In [5]:
getInfodata(datos)

Numero de postulaciones para obtener visa:  374362
Numero de características:  154


In [6]:
## Al parecer los atributos  "case_no" y "case_number", parecen ser lo mismo, por lo que 
## vamos a verificar esta situación
print("Primeras tres filas not null de case_no")
datos[datos.case_no.notnull()]['case_no'].head(3)

Primeras tres filas not null de case_no


0    A-07323-97014
1    A-07332-99439
2    A-07333-99643
Name: case_no, dtype: object

In [7]:
print("Últimas tres filas not null de case_no")
datos[datos.case_no.notnull()]['case_no'].tail(3)

Últimas tres filas not null de case_no


135266    A-13219-86523
135267    A-14267-09766
135268    A-14086-56170
Name: case_no, dtype: object

In [8]:
print("Primeras tres filas not null de case_number")
datos[datos.case_number.notnull()]['case_number'].head(3)

Primeras tres filas not null de case_number


135269    A-13316-14231
135270    A-13316-14287
135271    A-13316-14312
Name: case_number, dtype: object

In [9]:
print("Últimas filas not null de case_number")
datos[datos.case_number.notnull()]['case_number'].tail(3)

Últimas filas not null de case_number


374359    A-16354-82345
374360    A-16357-84250
374361    A-16279-59292
Name: case_number, dtype: object

Se observa que los valores faltantes de "case_no" son colocados en la columna "case_number". 
Por tanto, vamos añadir una columna "casenumber" obteniendo los valores de las columnas "case_no" y "case_number"

In [10]:
datacasenumber = []
idColumnCaseNro = datos.columns.get_loc("case_no")
idColumnCaseNumber = datos.columns.get_loc("case_number")


for valor in datos.iloc[0:135269, idColumnCaseNro]:
    datacasenumber.append(valor)
    
for valor in datos.iloc[135269:374362, idColumnCaseNumber]:
    datacasenumber.append(valor)
    
datos['casenumber'] = datacasenumber

In [11]:
datos = datos.drop(['case_no', 'case_number'], axis=1)

Ahora se va proseguir a observar el atributo "case_status", que es el que define si se da o no la visa. 

In [12]:
datos.case_status.value_counts()

Certified            181933
Certified-Expired    148586
Denied                25649
Withdrawn             18194
Name: case_status, dtype: int64

Los valores que no interesan, son aquellos con valor "Withdrawn", ya que este no me indica si se otorgó o no la visa. 

In [13]:
datos = datos[datos.case_status != 'Withdrawn']
datos.case_status.value_counts()

Certified            181933
Certified-Expired    148586
Denied                25649
Name: case_status, dtype: int64

Ahora los valores "Certified" y "Certified-Expired" indican lo mismo, es decir que se otorgo la visa.
Por tanto vamos a definir un sólo valor "Certified"

In [14]:
datos.loc[datos['case_status'] == 'Certified-Expired', 'case_status'] = 'Certified'
datos.case_status.value_counts()

Certified    330519
Denied        25649
Name: case_status, dtype: int64

Ahora proseguimos a eliminar aquellas filas y columnas que tengan valores perdidos mayor a un 85% de su total.

In [15]:
datos = datos.dropna(axis=0, how='all');
datos = datos.dropna(axis=1, how='all');
datos.shape

(356168, 153)

In [16]:
# Vamos a filtrar solo aquellas columnas que tienen al menos un 85% de valores no perdidos
numObservaciones = len(datos)*0.85
datos = datos.loc[:, datos.count() >= numObservaciones]
datos.shape

(356168, 18)